In [1]:
!pip install ipywidgets

### Basic usage
First we need to import the `Scenario` class which will be the central point for managing a given.. scenario.
Here's how to use it

In [12]:
from powersimdata import Scenario
scenario = Scenario(403)

SCENARIO: test | EasternBase_2020_3

--> State
analyze
--> Loading grid
--> Loading ct


### Note on documentation
See the [module index](https://breakthrough-energy.github.io/docs/py-modindex.html) on our docs website for a full listing of the code.
There is also interactive help available:

In [6]:
# see the docstring
scenario?

Type:           Scenario
String form:    <powersimdata.scenario.scenario.Scenario object at 0x7f654904ed30>
File:           /PowerSimData/powersimdata/scenario/scenario.py
Docstring:     
Handles scenario.

:param int/str descriptor: scenario name or index. If None, default to a Scenario
    in Create state.
Init docstring: Constructor.


In [4]:
scenario.print_scenario_info()

--------------------
SCENARIO INFORMATION
--------------------
id: 403
plan: test
name: EasternBase_2020_3
state: analyze
grid_model: usa_tamu
interconnect: Eastern
base_demand: vJan2021
base_hydro: vJan2021
base_solar: vJan2021
base_wind: vJan2021
change_table: Yes
start_date: 2016-01-01 00:00:00
end_date: 2016-12-31 23:00:00
interval: 24H
engine: REISE
runtime: 18:21
infeasibilities: 


### Load base scenario grid data and results

In [ ]:
ct= scenario.get_ct()
grid = scenario.get_grid()
load_shed= scenario.get_load_shed()
congu= scenario.get_congu() # get positive direction hourly branch congestion
congl= scenario.get_congl() # get negative direction hourly branch congestion

In [17]:
scenario.get_solar()

--> Loading solar


EmptyDataError: No columns to parse from file

### Congestion analysis

In [13]:
congu_yearly = (congu.sum(axis = 0)[congu.sum(axis = 0)!=0]).sort_values(ascending=False) # annual positive direction congestion
congl_yearly = (congl.sum(axis = 0)[congl.sum(axis = 0)!=0]).sort_values(ascending=False) # annual negative direction congestion
congl_yearly.at[68928] # get annual congestion on branch 68928
congl[68928] # get hourly congestion on branch 68928
congl[68928].astype(bool).sum(axis=0) # get annual total binding hours on 68928
grid.branch.loc[68928] # review information of branch 68928
grid.bus.loc[54070] # review from bus information of branch 68928
grid.bus.loc[54057] # review to bus information of branch 68928

type                  1
Pd                  0.0
Qd                  0.0
Gs                  0.0
Bs                  0.0
zone_id              39
Vm              1.03869
Va              16.9273
baseKV             69.0
loss_zone             1
Vmax                1.1
Vmin                0.9
lam_P             63.95
lam_Q               0.0
mu_Vmax             0.0
mu_Vmin             0.0
interconnect    Eastern
lat             42.3388
lon            -95.2962
Name: 54057, dtype: object

### Retrive other types of output

In [ ]:
pf_ac = scenario.get_pf() # get hourly generation output on units
pg_storage = scenario.get_storage_pg() # get generation profile for storage (if present in scenario)
e_storage = scenario.get_storage_e() # get energy state of charge of storage (if present in scenario)
pf_ac = scenario.get_pf() # get hourly power flow for AC lines
pf_dc = scenario.get_dcline_pf() # get hourly power flow for DC lines
lmp = scenario.get_lmp() # get hourly bus locational marginal price

--> Loading PF


### Create scenarios and upgrade congested transmission line

In [7]:
from powersimdata import Scenario
scenario = Scenario('')
scenario.set_builder(grid_model="usa_tamu", interconnect="Eastern")
scenario.set_name("MISONC_MTEP16_MEP", "training")
scenario.set_time("2016-01-01 00:00:00","2016-01-31 23:00:00","24H")
scenario.set_base_profile("demand", "vJan2021")
scenario.set_base_profile("hydro", "vJan2021")
scenario.set_base_profile("solar", "vJan2021")
scenario.set_base_profile("wind", "vJan2021")

scenario.change_table.scale_branch_capacity(branch_id = {68928: 17.28})

grid = scenario.get_grid()
ct = scenario.get_ct()


Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
--> Summary
# Existing study
test
# Available profiles
demand: vJan2021
hydro: vJan2021
solar: vJan2021
wind: vJan2021


### Use the change table to modify conditions

In [9]:
scenario.change_table.scale_plant_capacity(
    "solar", zone_name={"Arkansas": 5, "Louisiana": 2.5})
scenario.change_table.scale_plant_capacity(
    "wind", zone_name={"Iowa": 1.5, "Minnesota Southern": 2})
scenario.change_table.scale_branch_capacity(
    zone_name={"Iowa": 2, "Minnesota Southern": 2})
scenario.change_table.add_branch(
    [{"capacity": 236, "from_bus_id": 54070, "to_bus_id": 54057}])
scenario.change_table.add_dcline(
    [{"capacity": 2000, "from_bus_id":55119, "to_bus_id": 1213}])
scenario.change_table.add_plant(
    [{"type": "solar", "bus_id": 1899, "Pmax": 75},
     {"type": "coal", "bus_id": 54070, "Pmin": 25, "Pmax": 750, "c0": 1800, "c1": 30, "c2": 0.0025},
     {"type": "ng", "bus_id": 60911, "Pmax": 75, "c0": 900, "c1": 30, "c2": 0.0015}])
scenario.change_table.add_bus(
	[{"lat": 35, "lon": -90, "zone_id": 42, "baseKV": 500}])
scenario.change_table.add_dcline(
	[{"from_bus_id": 1899, "to_bus_id": 70048, "Pmin": 0, "Pmax": 200}])


### Review the scenario and change table, the launch simulation

In [10]:
grid = scenario.get_grid()
ct = scenario.get_ct()
scenario.print_scenario_info()

# scenario.create_scenario()

# print(scenario.state.name)
# scenario.state.print_scenario_status()

# from powersimdata.scenario.scenario import Scenario
# scenario = Scenario("training")
# scenario.print_scenario_info()
# scenario.state.prepare_simulation_input()

# process_run = scenario.state.launch_simulation(threads=8)
# process_run = scenario.state.launch_simulation(extract_data=False)


# process_run = scenario.state.launch_simulation()
# scenario.state.print_scenario_status()

# process_extract = scenario.state.extract_simulation_output()



--------------------
SCENARIO INFORMATION
--------------------
plan: MISONC_MTEP16_MEP
name: training
state: create
grid_model: usa_tamu
interconnect: Eastern
base_demand: vJan2021
base_hydro: vJan2021
base_solar: vJan2021
base_wind: vJan2021
change_table: Yes
start_date: 2016-01-01 00:00:00
end_date: 2016-01-31 23:00:00
interval: 24H
engine: REISE.jl


In [ ]:
pg = scenario.state.get_pg()
pf_ac = scenario.state.get_pf()
pf_dc = scenario.state.get_dcline_pf()
lmp = scenario.state.get_lmp()

cc_l = pf_ac * congl
cc_u = pf_ac * congu

cc_l_yearly = (cc_l.sum(axis = 0)[cc_l.sum(axis = 0)!=0]).sort_values(ascending=False)

cc_u_yearly = (cc_u.sum(axis = 0)[cc_u.sum(axis = 0)!=0]).sort_values(ascending=False)


In [ ]:
from powersimdata.design.transmission.mwmiles import calculate_mw_miles
calculate_mw_miles(scenario)

from powersimdata.design.transmission.statelines import classify_interstate_intrastate
classify_interstate_intrastate(scenario)

from powersimdata.design.generation.cost_curves import get_supply_data
supply_df = get_supply_data(grid,1)

from powersimdata.design.generation.cost_curves import build_supply_curve
P, F = P, F = build_supply_curve(grid, 1, 'Louisiana', 'ng’)

from powersimdata.design.generation.cost_curves import ks_test
max_diff = ks_test(P, F, P, F, 'Louisiana', 'ng')
max_diff


In [ ]:
from powersimdata.design.generation.cost_curves import plot_c1_vs_c2
plot_c1_vs_c2(grid, 'Louisiana', 'ng’)

In [ ]:
from powersimdata.design.generation.cost_curves import plot_capacity_vs_price
plot_capacity_vs_price(grid, 1, 'Louisiana', 'ng')

### More utilities in PostREISE

In [ ]:
from powersimdata.scenario.scenario import Scenario
scenario = Scenario(403) 
grid = scenario.state.get_grid()
pf_ac = scenario.state.get_pf()

from postreise.analyze.transmission.congestion import *
cs = calculate_congestion_surplus(scenario)

from postreise.analyze.transmission.utilization import *
cong_all = generate_cong_stats(pf_ac, grid.branch)
cong_all.head(n=20)
ut_df = get_utilization(grid.branch, pf_ac, median = True)

from postreise.plot.plot_utilization_map import *
map_utilization(ut_df,grid.branch)

SCENARIO: test | EasternBase_2020_3

--> State
analyze
--> Loading grid
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading dcline
Loading sub
Loading bus2sub
--> Loading ct
--> Loading PF


In [1]:
from postreise.analyze.generation.emissions import *
emissions = generate_emissions_stats(scenario)
emissions_bus = summarize_emissions_by_bus(emissions, grid)

from postreise.analyze.generation.curtailment import *
cur = calculate_curtailment_time_series(scenario)
calculate_curtailment_time_series_by_resources(scenario, resources={"solar", "wind"})
calculate_curtailment_time_series_by_areas(scenario, areas={"interconnect": {"Eastern"}, "state": {"Iowa", "Minnesota"}, "loadzone": {"Iowa", "Minnesota Southern"}})
calculate_curtailment_time_series_by_areas_and_resources(scenario, areas={"interconnect": {"Eastern"}, "state": {"Iowa"}, "loadzone": {"Iowa"}}, resources={"solar", "wind"})
cur.max()
cur.max().max()
cur.at['2016-01-04 19:00:00', 32]
summarize_curtailment_by_bus(scenario)
calculate_curtailment_percentage_by_resources(scenario, resources={'wind'})


NameError: name 'scenario' is not defined

In [ ]:
from postreise.plot.plot_utilization_map import *
map_utilization(ut_df,grid.branch)
